In [1]:
import pandas as pd
import numpy as np
import datetime
import requests
from portfoliomaker import createPortfolio 
import matplotlib.pyplot as plt
import json
from itertools import islice

In [2]:
with open('SnP500.txt', 'r') as f:
    SnP = ''.join(f.readlines()).split()

In [ ]:
identifiers = "/".join(SnP)

In [3]:
data = {}
for stock in SnP:
    portfolioAnalysisRequest = requests.get("https://www.blackrock.com/tools/hackathon/performance?" \
                 + "betaPortfolios=SNP500&endDate=20171231&identifiers="+stock \
                 + "&riskFreeRatePortfolio=LTBILL1-3M&startDate=20170101").json()
    data[stock] = portfolioAnalysisRequest

In [9]:
with open('data.json', 'w') as outfile: 
    json.dump(data, outfile)

In [ ]:
failed = []
for stock in SnP:
    if stock not in data.keys():
        failed.append(stock)
        SnP.remove(stock)

In [11]:
fig, ax = plt.subplots(figsize=(60,30))
for stock in SnP:
    if data[stock]['success'] == True:
        Y = [y[1] for y in islice(data[stock]['resultMap']['RETURNS'][0]['performanceChart'], 0, None, 1)]
        X = list(range(len(Y)))
        plt.plot(X,Y, label=stock)
        
plt.legend(prop={'size': 30})
plt.savefig('SnP.png')

In [4]:
stock_info = {}
for stock in SnP:
    if data[stock]['success'] == True:
        SecurityDataRequest = requests.get("https://www.blackrock.com/tools/hackathon/security-data?identifiers="+stock).json()
    data[stock]['resultMap']['SECURITY'] = SecurityDataRequest['resultMap']['SECURITY']

In [ ]:
from collections import defaultdict
sectors = defaultdict(list)
for stock in SnP:
    if data[stock]['success'] == True:
        if 'gics1Sector' in data[stock]['resultMap']['SECURITY'][0]:
            sector = data[stock]['resultMap']['SECURITY'][0]['gics1Sector']
            sectors[sector].append(stock)

In [ ]:
fig, ax1 = plt.subplots(3,4,figsize=(80,60))
ax=[]
for a in ax1:
    for i in a:
        ax.append(i)
for i, sector in enumerate(sectors.keys()):
    for stock in SnP:
        if data[stock]['success'] == True:
            if 'gics1Sector' in data[stock]['resultMap']['SECURITY'][0]:
                if data[stock]['resultMap']['SECURITY'][0]['gics1Sector'] == sector:
                    Y = [y[1] for y in islice(data[stock]['resultMap']['RETURNS'][0]['performanceChart'], 0, None, 1)]
                    X = list(range(len(Y)))
                    ax[i].plot(X, Y, linewidth=4)
        
        plt.yticks(size=100)
        plt.xticks(size=100, rotation=45)
        ax[i].set_title(sector)
        ax[i].legend(prop={'size': 100})
plt.tight_layout()
plt.savefig('sectors.png')